# **AI LiDAR Load in Program**

### **Import Librararies:**

Ensure when importing the Libraries that you have already installed them  

In [ ]:
import os
import zipfile
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

## **Extract The Local Zip Files:**

This progarm first analzyes whether not a folder exists or not, and creates the folder if it didn't already exist. Then you extract the files in the directory and have a progress bar for tracking its progress


In [ ]:
def extract_local_zip(zip_path, extract_path):
    # Check if the zip file exists
    if not os.path.exists(zip_path):
        print(f"Error: Zip file not found at {zip_path}")
        return

    # Create the extraction directory if it doesn't exist
    if not os.path.exists(extract_path):
        os.makedirs(extract_path)

    try:
        # Extract the contents of the zip file with tqdm progress bar
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # Handle the case where the zip file contains a single directory
            file_list = zip_ref.namelist()
            for file in tqdm(file_list, desc="Extracting"):
                zip_ref.extract(file, extract_path)

        print("Zip file extracted successfully.")
    except Exception as e:
        print(f"Error extracting zip file: {e}")


## **Augment the Dataset for Usage**

First we create the folder for the augmented_folder and if it already exists,ignores the creation. Then we format the images use the ImageDataGenrator function. Then we add the funal touches the generate the data, and format it perfectly for AI. FInally the tqdm bar is defined



In [ ]:
def create_and_augment_dataset(extract_path, augmented_path, target_image_size=(224, 224), batch_size=16, total_augmented_images=None):
    # Create the augmented images directory if it doesn't exist
    if not os.path.exists(augmented_path):
        os.makedirs(augmented_path)

    # Create an ImageDataGenerator with your desired parameters
    datagen = ImageDataGenerator(
        rotation_range=45,
        zoom_range=0.2,
        horizontal_flip=True,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.2,
        rescale=1./255  # Added rescaling to normalize pixel values
    )

    # Create a generator from the data in the extracted path
    generator = datagen.flow_from_directory(
        extract_path,
        target_size=target_image_size,
        batch_size=batch_size,
        class_mode='categorical',
        save_to_dir=augmented_path,  # Save the augmented images to the specified directory
        save_prefix='augmented',
        save_format='png'
    )

    # Determine the total number of images to augment
    total_images = min(total_augmented_images, generator.samples) if total_augmented_images else generator.samples

    # Create the augmented images with tqdm progress bar
    for _ in tqdm(range(total_images), desc="Augmenting"):
        try:
            next(generator)
        except StopIteration:
            break  # Stop if the generator has no more data

    print(f"{total_images} images in the folder augmented and saved successfully.")


## **Main Program/ Running**
We define our three paths and run the functions

In [ ]:
# Example usage:
zip_file = r"C:\\Users\\allar\\Downloads\\data_object_image_2.zip"
extracted_path = r"C:\\Users\\allar\\Downloads\\extracted_data"  # Update the directory path
augmented_path = r"C:\\Users\\allar\\Downloads\\augmented_data"  # Update the directory path

# Extract the contents of the zip file
extract_local_zip(zip_file, extracted_path)

# Create and augment the dataset
create_and_augment_dataset(extracted_path, augmented_path, target_image_size=(224, 224), batch_size=16, total_augmented_images=None)
